<a href="https://colab.research.google.com/github/sheilallee/dataset/blob/main/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de Machine Learning: Predição de Casos de COVID-19 em Síndrome Gripal (e-SUS Notifica/PB)

## Finalidade do Projeto:

O propósito central deste projeto é aplicar técnicas de **Machine Learning (Aprendizado de Máquina)** para construir um modelo preditivo. O objetivo é auxiliar na vigilância epidemiológica, estimando a probabilidade de um paciente notificado com Síndrome Gripal (SG) ser classificado como **Caso Confirmado de COVID-19** através dos sintomas, utilizando apenas as informações demográficas e clínicas registradas no início da notificação.

## Problema de Machine Learning

Este projeto é um problema de **Classificação Binária**.

* **Pergunta Central:** Dado o perfil de um paciente (sintomas, idade, condições pré-existentes, etc.), podemos prever se a sua classificação final será **COVID-19 Positivo** ou **Negativo/Descartado**?
* **Variável-Alvo (`Y`):** A coluna `classificacaoFinal` será o alvo da predição, por meio da transformação em uma variável binária que indique se o caso foi confirmado para COVID-19 (1) ou não (0). Essa será a variável-alvo (Y) usada posteriormente no modelo de classificação.

## Descrição do Dataset

O conjunto de dados utilizado é proveniente do **Ministério da Saúde (DATASUS)**, especificamente das **Notificações de Síndrome Gripal (SG)** registradas na Paraíba (PB) no ano de 2024.

* **Origem:** Sistema **e-SUS Notifica**, implementado para a vigilância de casos leves e moderados suspeitos de COVID-19 a partir de março de 2020.
* **Conteúdo:** A base contém informações detalhadas de cada paciente notificado, incluindo dados demográficos (`idade`, `sexo`, `racaCor`), clínicos (`sintomas`, `outrasCondicoes`) e informações logísticas da notificação (`municipioNotificacao`, `dataInicioSintomas`).
* **Relevância:** Por ser um dado real do Sistema Único de Saúde (SUS), o dataset é de grande volume (cerca de 87917 mil registros) e apresenta as complexidades e os desafios de bases de dados de saúde pública, como a alta taxa de valores faltantes (`NaN`) e a necessidade de padronização de codificação.
* **Limitações dos dados:** Estados e municípios que utilizam sistemas próprios de notificação de casos suspeitos de covid-19 estão em processo de integração com o e-SUS Notifica. Assim, os dados desses locais podem apresentar uma diferença substancial até a finalização do processo de integração.

## Fonte

* **Dados:** https://opendatasus.saude.gov.br/dataset/notificacoes-de-sindrome-gripal-leve-2024
* **Dicionário de Dados:** https://opendatasus.saude.gov.br/dataset/notificacoes-de-sindrome-gripal-leve-2024/resource/17ede700-ab05-4233-9e4d-c28c768e759f
* **Dataset no github:** https://github.com/sheilallee/dataset.git




In [ ]:
#CONFIGURAÇÃO E CARREGAMENTO DO DATASET
# Instala pacote (rodar 1x)
!pip install Unidecode

import pandas as pd
import seaborn as srn
import statistics as sts
import numpy as np
import re
from unidecode import unidecode

#CONFIGURAÇÃO DE EXIBIÇÃO DO PANDAS
pd.set_option('display.max_columns', None)

# Caminho completo para o arquivo
file_path = 'https://raw.githubusercontent.com/sheilallee/dataset/refs/heads/main/dadosPB.csv'

# Carregando o dataset
dataset = pd.read_csv(file_path, delimiter=';', encoding='utf-8')
# Exibindo as primeiras linhas
dataset.head(10)


/tmp/ipython-input-3835731396.py:19: DtypeWarning: Columns (12,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(file_path, delimiter=';', encoding='utf-8')


,sintomas,profissionalSaude,racaCor,outrosSintomas,outrasCondicoes,profissionalSeguranca,cbo,condicoes,sexo,estado,estadoIBGE,municipio,municipioIBGE,origem,estadoNotificacao,municipioNotificacao,municipioNotificacaoIBGE,evolucaoCaso,classificacaoFinal,codigoEstrategiaCovid,codigoBuscaAtivaAssintomatico,outroBuscaAtivaAssintomatico,codigoTriagemPopulacaoEspecifica,outroTriagemPopulacaoEspecifica,codigoLocalRealizacaoTestagem,outroLocalRealizacaoTestagem,codigoRecebeuVacina,codigoLaboratorioPrimeiraDose,codigoLaboratorioSegundaDose,lotePrimeiraDose,loteSegundaDose,codigoContemComunidadeTradicional,source_id,excluido,validado,codigoDosesVacina,estadoNotificacaoIBGE,totalTestesRealizados,dataNotificacao,dataInicioSintomas,dataEncerramento,dataPrimeiraDose,dataSegundaDose,codigoEstadoTeste1,codigoTipoTeste1,codigoFabricanteTeste1,codigoResultadoTeste1,codigoEstadoTeste2,codigoTipoTeste2,codigoFabricanteTeste2,codigoResultadoTeste2,codigoEstadoTeste3,codigoTipoTeste3,codigoFabricanteTeste3,codigoResultadoTeste3,codigoEstadoTeste4,codigoTipoTeste4,codigoFabricanteTeste4,codigoResultadoTeste4,dataColetaTeste1,dataColetaTeste2,dataColetaTeste3,dataColetaTeste4,idade
0,"Dor de Cabeça, Tosse, Febre, Dor de Garganta",Não,Branca,NaN,NaN,Não,NaN,Diabetes,Masculino,Pernambuco,PE,Recife,2611606,parse-cloud,Paraíba,João Pessoa,2507507.0,Cura,Síndrome Gripal Não Especificada,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,ASTRAZENECA/FIOCRUZ,NaN,214VCD085W,NaN,2.0,MTCKsWDZ7n,False,False,"1,2,3",PB,1,2022-11-15,2022-11-13,2024-12-03,2021-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Coriza, Tosse",Não,Parda,NaN,NaN,Não,NaN,NaN,Masculino,Paraíba,PB,São Bento,2513901,parse-cloud,Paraíba,São Bento,2513901.0,Ignorado,Descartado,1.0,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,2.0,T4yRLL1L8f,False,False,NaN,PB,2,2022-06-30,2022-06-25,2024-01-30,NaN,NaN,3.0,3.0,792.0,2.0,3.0,3.0,715.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-29,2022-06-30,NaN,NaN,23.0
2,Assintomático,Não,Ignorado,NaN,NaN,Não,NaN,NaN,Masculino,Paraíba,PB,Soledade,2516102,parse-cloud,Paraíba,Soledade,2516102.0,Cura,Confirmado Laboratorial,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,SINOVAC/BUTANTAN,SINOVAC/BUTANTAN,202009011,202010028,2.0,oKrbuCdoED,False,False,"2,1",PB,5,2022-01-20,NaN,2024-03-23,2021-01-25,2021-02-15,3.0,3.0,715.0,2.0,3.0,3.0,715.0,1.0,3.0,3.0,715.0,1.0,3.0,3.0,792.0,1.0,2024-03-23,2023-03-20,2022-11-25,2022-06-28,50.0
3,"Dor de Cabeça, Dor de Garganta",Não,Parda,NaN,NaN,Não,NaN,NaN,Feminino,Pernambuco,PE,Recife,2611606,parse-cloud,Paraíba,João Pessoa,2507507.0,Cura,Síndrome Gripal Não Especificada,1.0,NaN,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,2.0,b9SDxlsKX3,False,False,NaN,PB,1,2022-03-02,2022-02-07,2024-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.0
4,"Dor de Cabeça, Dor de Garganta, Tosse",Sim,Amarela,NaN,NaN,Não,3222 - Técnicos e auxiliares de enfermagem,NaN,Feminino,Paraíba,PB,João Pessoa,2507507,parse-cloud,Paraíba,João Pessoa,2507507.0,Cura,Síndrome Gripal Não Especificada,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,SINOVAC/BUTANTAN,SINOVAC/BUTANTAN,202009011,202009011,2.0,Y1HdujjH8Z,False,False,"1,2,3,4",PB,1,2022-12-27,2022-12-24,2024-01-30,2021-01-20,2021-02-16,3.0,3.0,915.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-12-27,NaN,NaN,NaN,42.0
5,"Febre, Dor de Garganta",Não,Parda,NaN,NaN,Não,NaN,NaN,Masculino,Pernambuco,PE,Recife,2611606,parse-cloud,Paraíba,João Pessoa,2507507.0,Cura,Síndrome Gripal Não Especificada,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,PFIZER,NaN,FF8843,NaN,2.0,DRe21VD9dA,False,False,1,PB,1,2022-06-06,2022-05-18,2024-12-03,2021-09-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0
6,"Coriza, Dor de Garganta",Não,Branca,NaN,NaN,Não,NaN,NaN,Feminino,Pernambuco,PE,Recife,2611606,parse-cloud,Paraíba,Campina Grande,2504009.0,Cura,Síndrome Gripal Não Especificada,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,SINOVAC/BUTANTAN,NaN,210078,NaN,2.0,Ipydw

## Listagem de Colunas e Classificar Manualmente



*   colunas_candidatas (demográficas, clínicas)
*   colunas_para_remover (administrativas, pós-notificação, vacinas, códigos específicos)



## Filtragem de Dados: Exclusão de Colunas

Foram removidas algumas colunas do conjunto de dados (dataset) referentes a informações pós-notificação, detalhes de vacinação (códigos laboratoriais) e campos meramente administrativos, tais como: codigoBuscaAtivaAssintomatico, codigoLocalRealizacaoTestagem, outroLocalRealizacaoTestagem, e codigoContemComunidadeTradicional.

A justificativa para a retirada desses campos é que eles não descrevem o perfil clínico do paciente ou seus sintomas. São, em essência, informações operacionais do sistema de vigilância epidemiológica, não contribuindo para a modelagem preditiva baseada nas características dos casos.

Desta forma, reduzimos a base apenas às colunas que representam informações clínicas e demográficas presentes no início da notificação. Removemos dados laboratoriais, datas posteriores e colunas irrelevantes para evitar vazamento de dados e melhorar a qualidade da modelagem.

In [ ]:
#FILTRAGEM DE COLUNAS

#COLUNAS SELECIONADAS
colunas_desejadas = [
    'sintomas', 'outrosSintomas',
    'sexo','idade','racaCor',
    'municipioNotificacao','estadoNotificacaoIBGE',
    'evolucaoCaso',
    'totalTestesRealizados',
    'dataNotificacao',
    'classificacaoFinal'
]

# CRIAÇÃO DO DATASET FILTRADO
dataset_filtrado = dataset[colunas_desejadas].copy()

dataset_filtrado.head(10)

,sintomas,outrosSintomas,sexo,idade,racaCor,municipioNotificacao,estadoNotificacaoIBGE,evolucaoCaso,totalTestesRealizados,dataNotificacao,classificacaoFinal
0,"Dor de Cabeça, Tosse, Febre, Dor de Garganta",NaN,Masculino,NaN,Branca,João Pessoa,PB,Cura,1,2022-11-15,Síndrome Gripal Não Especificada
1,"Coriza, Tosse",NaN,Masculino,23.0,Parda,São Bento,PB,Ignorado,2,2022-06-30,Descartado
2,Assintomático,NaN,Masculino,50.0,Ignorado,Soledade,PB,Cura,5,2022-01-20,Confirmado Laboratorial
3,"Dor de Cabeça, Dor de Garganta",NaN,Feminino,53.0,Parda,João Pessoa,PB,Cura,1,2022-03-02,Síndrome Gripal Não Especificada
4,"Dor de Cabeça, Dor de Garganta, Tosse",NaN,Feminino,42.0,Amarela,João Pessoa,PB,Cura,1,2022-12-27,Síndrome Gripal Não Especificada
5,"Febre, Dor de Garganta",NaN,Masculino,20.0,Parda,João Pessoa,PB,Cura,1,2022-06-06,Síndrome Gripal Não Especificada
6,"Coriza, Dor de Garganta",NaN,Feminino,NaN,Branca,Campina Grande,PB,Cura,1,2022-12-09,Síndrome Gripal Não Especificada
7,"Coriza, Dor de Cabeça, Tosse, Dor de Garganta",NaN,Masculino,52.0,Parda,João Pessoa,PB,Cura,1,2022-11-23,Síndrome Gripal Não Especificada
8,"Coriza, Distúrbios Olfativos, Tosse, Dispneia,...",TONTURA,Feminino,23.0,Amarela,Aroeiras,PB,Cura,1,2022-02-10,Confirmado Laboratorial
9,"Coriza, Dor de Cabeça, Dor de Garganta",NaN,Feminino,NaN,Ignorado,Santa Rita,PB,Cura,1,2022-07-26,Confirmado Laboratorial


## Limpeza de Dados

* Padronizar a formatação das colunas e linhas:
  * Aplicar letras minúsculas (lowercase);
  * Remover caracteres especiais.
*   Verificar proporção de valores ausentes, para entender onde a base é mais incompleta e definir a estratégia de tratamento.
*   Substituir NaN por vazio nas colunas de texto;
* Tratamento da Idade, em virtude de ter valores zerados e vazios:
  * Converter para número;
  * Valores <= 0 e NaN;
  * Imputar com mediana;
  * *Usar a discretização para trasnsformar em variáveis categóricas.*


In [ ]:
# Tratamento de Texto: Preencher NaN com String Vazia
colunas_texto = dataset_filtrado.select_dtypes(include=["object"]).columns
dataset_filtrado.loc[:, colunas_texto] = dataset_filtrado[colunas_texto].fillna("")

# Definir explicitamente quais colunas são datas
colunas_data = [c for c in ['DATANOTIFICACAO'] if c in dataset_filtrado.columns]

# Converter para datetime
def to_dt(v):
    if pd.isna(v):
        return pd.NaT
    s = str(v).strip()
    if not s:
        return pd.NaT
    # Formato ISO já com hífen
    if re.fullmatch(r'\d{4}-\d{2}-\d{2}', s):
        return pd.to_datetime(s, format='%Y-%m-%d', errors='coerce')
    # Formato compacto
    if re.fullmatch(r'\d{8}', s):
        return pd.to_datetime(s, format='%Y%m%d', errors='coerce')
    # Tenta parse genérico
    return pd.to_datetime(s, errors='coerce')

for c in colunas_data:
    # Só converte se ainda não for datetime
    if dataset_filtrado[c].dtype == 'object':
        dataset_filtrado[c] = dataset_filtrado[c].apply(to_dt)

# 4. Formatação apenas para exibição
format_dict = {
    c: (lambda x: x.strftime("%Y-%m-%d") if pd.notna(x) else "")
    for c in colunas_data
}

display(dataset_filtrado.head(10).style.format(format_dict))

,sintomas,outrosSintomas,sexo,idade,racaCor,municipioNotificacao,estadoNotificacaoIBGE,evolucaoCaso,totalTestesRealizados,dataNotificacao,classificacaoFinal
0,"Dor de Cabeça, Tosse, Febre, Dor de Garganta",,Masculino,nan,Branca,João Pessoa,PB,Cura,1,2022-11-15,Síndrome Gripal Não Especificada
1,"Coriza, Tosse",,Masculino,23.000000,Parda,São Bento,PB,Ignorado,2,2022-06-30,Descartado
2,Assintomático,,Masculino,50.000000,Ignorado,Soledade,PB,Cura,5,2022-01-20,Confirmado Laboratorial
3,"Dor de Cabeça, Dor de Garganta",,Feminino,53.000000,Parda,João Pessoa,PB,Cura,1,2022-03-02,Síndrome Gripal Não Especificada
4,"Dor de Cabeça, Dor de Garganta, Tosse",,Feminino,42.000000,Amarela,João Pessoa,PB,Cura,1,2022-12-27,Síndrome Gripal Não Especificada
5,"Febre, Dor de Garganta",,Masculino,20.000000,Parda,João Pessoa,PB,Cura,1,2022-06-06,Síndrome Gripal Não Especificada
6,"Coriza, Dor de Garganta",,Feminino,nan,Branca,Campina Grande,PB,Cura,1,2022-12-09,Síndrome Gripal Não Especificada
7,"Coriza, Dor de Cabeça, Tosse, Dor de Garganta",,Masculino,52.000000,Parda,João Pessoa,PB,Cura,1,2022-11-23,Síndrome Gripal Não Especificada
8,"Coriza, Distúrbios Olfativos, Tosse, Dispneia, Outros, Dor de Cabeça",TONTURA,Feminino,23.000000,Amarela,Aroeiras,PB,Cura,1,2022-02-10,Confirmado Laboratorial
9,"Coriza, Dor de Cabeça, Dor de Garganta",,Feminino,nan,Ignorado,Santa Rita,PB,Cura,1,2022-07-26,Confirmado Laboratorial


In [ ]:
# Garantir coluna de data como datetime
colunas_data = ['DATANOTIFICACAO']

def para_datetime(valor):
    if pd.isna(valor):
        return pd.NaT
    if isinstance(valor, pd.Timestamp):
        return valor
    s = str(valor).strip()
    if not s:
        return pd.NaT
    if re.fullmatch(r'\d{4}-\d{2}-\d{2}', s):
        return pd.to_datetime(s, format='%Y-%m-%d', errors='coerce')
    if re.fullmatch(r'\d{8}', s):
        return pd.to_datetime(s, format='%Y%m%d', errors='coerce')
    return pd.to_datetime(s, errors='coerce')

for c in colunas_data:
    if c in dataset_filtrado.columns:
        dataset_filtrado[c] = dataset_filtrado[c].apply(para_datetime)

# Normalizar texto
colunas_alvo_base = [
    'sintomas',
    'outrosSintomas',
    'municipioNotificacao',
    'evolucaoCaso',
    'classificacaoFinal',
    'sexo',
    'racaCor',
    'estadoNotificacaoIBGE'
]

# Mapeia nomes reais existentes no DataFrame independentemente de maiúsculas/minúsculas
lookup = {c.lower(): c for c in dataset_filtrado.columns}
colunas_encontradas = [lookup[n.lower()] for n in colunas_alvo_base if n.lower() in lookup]

def normalizar_texto(valor):
    if isinstance(valor, str):
        return unidecode(valor).lower().strip()
    return valor

for c in colunas_encontradas:
    dataset_filtrado[c] = dataset_filtrado[c].apply(normalizar_texto)

display(dataset_filtrado.head(10))

,sintomas,outrosSintomas,sexo,idade,racaCor,municipioNotificacao,estadoNotificacaoIBGE,evolucaoCaso,totalTestesRealizados,dataNotificacao,classificacaoFinal
0,"dor de cabeca, tosse, febre, dor de garganta",,masculino,NaN,branca,joao pessoa,pb,cura,1,2022-11-15,sindrome gripal nao especificada
1,"coriza, tosse",,masculino,23.0,parda,sao bento,pb,ignorado,2,2022-06-30,descartado
2,assintomatico,,masculino,50.0,ignorado,soledade,pb,cura,5,2022-01-20,confirmado laboratorial
3,"dor de cabeca, dor de garganta",,feminino,53.0,parda,joao pessoa,pb,cura,1,2022-03-02,sindrome gripal nao especificada
4,"dor de cabeca, dor de garganta, tosse",,feminino,42.0,amarela,joao pessoa,pb,cura,1,2022-12-27,sindrome gripal nao especificada
5,"febre, dor de garganta",,masculino,20.0,parda,joao pessoa,pb,cura,1,2022-06-06,sindrome gripal nao especificada
6,"coriza, dor de garganta",,feminino,NaN,branca,campina grande,pb,cura,1,2022-12-09,sindrome gripal nao especificada
7,"coriza, dor de cabeca, tosse, dor de garganta",,masculino,52.0,parda,joao pessoa,pb,cura,1,2022-11-23,sindrome gripal nao especificada
8,"coriza, disturbios olfativos, tosse, dispneia,...",tontura,feminino,23.0,amarela,aroeiras,pb,cura,1,2022-02-10,confirmado laboratorial
9,"coriza, dor de cabeca, dor de garganta",,feminino,NaN,ignorado,santa rita,pb,cura,1,2022-07-26,confirmado laboratorial


In [ ]:
#idades fora do domínio
dataset_filtrado.loc[(dataset_filtrado['idade'] <= 0) | (dataset_filtrado['idade'] > 110)]
print("Total de idades inválidas (<=0 ou >110):", idades_invalidas.shape[0])
display(idades_invalidas.head(10))



Total de idades inválidas (<=0 ou >110): 4461


,SINTOMAS,OUTROSSINTOMAS,SEXO,IDADE,RACACOR,MUNICIPIONOTIFICACAO,ESTADONOTIFICACAOIBGE,EVOLUCAOCASO,TOTALTESTESREALIZADOS,DATAINICIOSINTOMAS,DATANOTIFICACAO,CLASSIFICACAOFINAL
1666,"CORIZA, DOR DE CABECA",,FEMININO,0.0,PARDA,PRINCESA ISABEL,PB,CURA,1,2023-12-15,2023-12-18,CONFIRMADO LABORATORIAL
1680,"CORIZA, TOSSE, FEBRE",,FEMININO,0.0,PARDA,PEDRAS DE FOGO,PB,CURA,1,2023-12-10,2023-12-14,CONFIRMADO LABORATORIAL
1697,"CORIZA, DISTURBIOS GUSTATIVOS, DOR DE CABECA, ...",,FEMININO,0.0,IGNORADO,JOAO PESSOA,PB,CURA,1,2023-12-04,2023-12-11,CONFIRMADO LABORATORIAL
1758,OUTROS,PRE OPERATORIO DE CIRURGIAS HRPI,FEMININO,0.0,PARDA,PRINCESA ISABEL,PB,CURA,1,2023-10-05,2023-10-05,DESCARTADO
1765,OUTROS,PRE OPERATORIO DE CIRURGIAS HRPI,MASCULINO,0.0,PARDA,PRINCESA ISABEL,PB,CURA,1,2023-12-12,2023-12-12,DESCARTADO
1880,"TOSSE, FEBRE, DISPNEIA",,MASCULINO,0.0,PARDA,SANTA RITA,PB,CURA,1,2023-09-09,2023-09-11,CONFIRMADO CLINICOEPIDEMIOLOGICO
1948,ASSINTOMATICO,,MASCULINO,0.0,PARDA,CAMPINA GRANDE,PB,CURA,1,NaT,2023-04-18,SINDROME GRIPAL NAO ESPECIFICADA
1952,"CORIZA, TOSSE, FEBRE",,MASCULINO,0.0,IGNORADO,JOAO PESSOA,PB,CURA,1,2023-12-18,2023-12-27,CONFIRMADO LABORATORIAL
1968,"TOSSE, FEBRE, DOR DE GARGANTA",,FEMININO,0.0,IGNORADO,JOAO PESSOA,PB,CURA,1,2023-12-10,2023-12-13,CONFIRMADO LABORATORIAL
2093,"DOR DE CABECA, TOSSE, CORIZA",,FEMININO,0.0,IGNORADO,JOAO PESSOA,PB,CURA,1,2023-12-04,2023-12-12,CONFIRMADO LABORATORIAL


In [ ]:
#calcular a mediana da idade

# Garantir que idade seja numérica
dataset_filtrado['idade'] = pd.to_numeric(dataset_filtrado['idade'], errors='coerce')

# Máscara de idades válidas (intervalo 1–110)
mask_validas = (dataset_filtrado['idade'] > 0) & (dataset_filtrado['idade'] <= 110)

# Calcular mediana usando somente idades válidas
mediana_idade = dataset_filtrado.loc[mask_validas, 'idade'].median()

# Imputar idades inválidas com a mediana
dataset_filtrado.loc[~mask_validas & dataset_filtrado['idade'].notna(), 'idade'] = mediana_idade

# Imputar NaN com a mediana
dataset_filtrado['idade'] = dataset_filtrado['idade'].fillna(mediana_idade)

# Mediana
print(f"Mediana: {mediana_idade}")

display(dataset_filtrado.head(10))

Mediana: 25.0


,sintomas,outrosSintomas,sexo,idade,racaCor,municipioNotificacao,estadoNotificacaoIBGE,evolucaoCaso,totalTestesRealizados,dataNotificacao,classificacaoFinal
0,"dor de cabeca, tosse, febre, dor de garganta",,masculino,25.0,branca,joao pessoa,pb,cura,1,2022-11-15,sindrome gripal nao especificada
1,"coriza, tosse",,masculino,23.0,parda,sao bento,pb,ignorado,2,2022-06-30,descartado
2,assintomatico,,masculino,50.0,ignorado,soledade,pb,cura,5,2022-01-20,confirmado laboratorial
3,"dor de cabeca, dor de garganta",,feminino,53.0,parda,joao pessoa,pb,cura,1,2022-03-02,sindrome gripal nao especificada
4,"dor de cabeca, dor de garganta, tosse",,feminino,42.0,amarela,joao pessoa,pb,cura,1,2022-12-27,sindrome gripal nao especificada
5,"febre, dor de garganta",,masculino,20.0,parda,joao pessoa,pb,cura,1,2022-06-06,sindrome gripal nao especificada
6,"coriza, dor de garganta",,feminino,25.0,branca,campina grande,pb,cura,1,2022-12-09,sindrome gripal nao especificada
7,"coriza, dor de cabeca, tosse, dor de garganta",,masculino,52.0,parda,joao pessoa,pb,cura,1,2022-11-23,sindrome gripal nao especificada
8,"coriza, disturbios olfativos, tosse, dispneia,...",tontura,feminino,23.0,amarela,aroeiras,pb,cura,1,2022-02-10,confirmado laboratorial
9,"coriza, dor de cabeca, dor de garganta",,feminino,25.0,ignorado,santa rita,pb,cura,1,2022-07-26,confirmado laboratorial


In [ ]:
#Verificação Final de dados Faltantes
ver_final = dataset_filtrado.isna().mean().sort_values(ascending=False)
ver_final.head(20)

,0
sintomas,0.0
outrosSintomas,0.0
sexo,0.0
idade,0.0
racaCor,0.0
municipioNotificacao,0.0
estadoNotificacaoIBGE,0.0
evolucaoCaso,0.0
totalTestesRealizados,0.0
dataNotificacao,0.0


In [ ]:
dataset_filtrado.head(30)

,sintomas,outrosSintomas,sexo,idade,racaCor,municipioNotificacao,estadoNotificacaoIBGE,evolucaoCaso,totalTestesRealizados,dataNotificacao,classificacaoFinal
0,"dor de cabeca, tosse, febre, dor de garganta",,masculino,25.0,branca,joao pessoa,pb,cura,1,2022-11-15,sindrome gripal nao especificada
1,"coriza, tosse",,masculino,23.0,parda,sao bento,pb,ignorado,2,2022-06-30,descartado
2,assintomatico,,masculino,50.0,ignorado,soledade,pb,cura,5,2022-01-20,confirmado laboratorial
3,"dor de cabeca, dor de garganta",,feminino,53.0,parda,joao pessoa,pb,cura,1,2022-03-02,sindrome gripal nao especificada
4,"dor de cabeca, dor de garganta, tosse",,feminino,42.0,amarela,joao pessoa,pb,cura,1,2022-12-27,sindrome gripal nao especificada
5,"febre, dor de garganta",,masculino,20.0,parda,joao pessoa,pb,cura,1,2022-06-06,sindrome gripal nao especificada
6,"coriza, dor de garganta",,feminino,25.0,branca,campina grande,pb,cura,1,2022-12-09,sindrome gripal nao especificada
7,"coriza, dor de cabeca, tosse, dor de garganta",,masculino,52.0,parda,joao pessoa,pb,cura,1,2022-11-23,sindrome gripal nao especificada
8,"coriza, disturbios olfativos, tosse, dispneia,...",tontura,feminino,23.0,amarela,aroeiras,pb,cura,1,2022-02-10,confirmado laboratorial
9,"coriza, dor de cabeca, dor de garganta",,feminino,25.0,ignorado,santa rita,pb,cura,1,2022-07-26,confirmado laboratorial
